In [1]:
import os
from utils import helpers

os.chdir('..')

In [2]:
import hashlib

In [3]:
input = helpers.get_input(2016, 5)

Found a datafile!


In [4]:
print(hashlib.md5(b'abc5278568').hexdigest())

00000f9a2c309875e05c5a5d09f1b8c4


In [5]:
def abba_finder(string:str):
    for i, char in enumerate(string[:-3]):
        if string[i] == string[i + 3]:
            if string[i + 1] == string[i + 2]:
                if string[i] != string[i + 1]:
                    print(char + string[i + 1] + string[i+2] + string[i +3])
                    return True
    return False

In [8]:
print(abba_finder("hjgssdafsdf[adda]jksdhfkjddadsda"))

adda
True


In [9]:
a = "abcd"
b = a
a = "sasdsdsad"
print(b)

abcd


In [11]:
print(5 > 4 and 3 < 4)

True
